In [1]:
import sqlite3
import pandas as pd 
import numpy as np 

pd.set_option('max_colwidth', 300)
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 300)

## Class Version

In [3]:
def parse_callout(raw_str):
    str1 = raw_str.replace('$', '').replace(',', '')
    try:
        out = float(str1)
    except:
        out = np.nan
    return(out)

In [4]:
def api_gb_func(df):
    out = {}
    out['Count'] = df.shape[0]
    out['# w/ $ callout'] = df['callout_parse'].notnull().sum()
    if df.shape[0] > 1:
        out['Avg Callout Price'] = np.round(df['callout_parse'].mean(), 2)
        out['Avg Callout Pct Diff'] = np.round(df['callout_diff_pct'].mean(), 2)
    else:
        out['Avg Callout Price'] = np.nan
        out['Avg Callout Pct Diff'] = np.nan
    
    return(pd.Series(out))

In [8]:
class DealnewsAnalyzer():
    
    def __init__(self):
        self.db_path = '/home/malcolm/Dealnews/dealnews.db'
        pass 
    
    def create_con(self):
        self.con = sqlite3.connect(self.db_path)
        self.cursor = con.cursor()
    
    def check_db_stats(self):
        tables = self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
        tables = [x[0] for x in tables]
        print("Tables: ", tables)
        
        self.dn_items_schema = cursor.execute("pragma table_info(`dealnews items`)").fetchall()
        print("Dealnews items schema: ", dn_self.dn_items_schema)
        
        
    def get_items_table(self, where_clause=''):
        self.items = pd.read_sql('Select * from `Dealnews self.items` '+ where_clause, self.con)
        self.items['callout_parse'] = self.items['call_out'].apply(parse_callout)
        self.items['callout_compare_parse'] = self.items['call_out_comparison'].apply(parse_callout)
        parsed_cnt = self.items['callout_parse'].notnull().sum()
        parsed_pct = np.round(100*parsed_cnt/self.items.shape[0], 2)
        parse_compare_cnt = self.items['callout_compare_parse'].notnull().sum()
        print(f"Parsed: {parsed_cnt}, Parsed Pct: {parsed_pct}, Parse compare cnt: {parse_compare_cnt}")
        
    def filter_items(self):
        self.items_sm = self.items.drop_duplicates(subset=['headline'])
        self.items_sm = self.items_sm[self.items_sm['callout_parse'].notnull()]
        print(f"Keep {self.items_sm.shape[0]} ", np.round(100*self.items_sm.shape[0]/self.items.shape[0],2))
        
    def get_category_table(self):
        last_category_date = cursor.execute("select max(_dt_pulled) from `Category Info`")
        last_category_date = last_category_date.fetchall()[0][0]
        print(last_category_date)
        categories = pd.read_sql(f"select * from `Category Info` where _dt_pulled='{last_category_date}' ", con)
        category_ids_sm = categories[categories['level_0'] == 'category']
        category_ids_sm = category_ids_sm[['id_number', 'name', 'short_name']]
        
    def combine_items_category_names(self):
        self.items2 = pd.merge(self.items_sm, category_ids_sm
                  , left_on = 'category_id_0', right_on='id_number'
                  , how='left').drop(['id_number', 'short_name'], axis=1)\
            .rename({'name_x':'vendor name', 
                     'name_y':'category name'}, axis=1)
    
    def top_vendors(self):
        vendor_gb = self.items2.groupby('vendor name').apply(lambda x:api_gb_func(x))\
            .sort_values('Count', ascending=False)
        top_vendors = vendor_gb.sort_values('Count', ascending=False)[:10].index.tolist()
   
    def category_gb(self):
        cateogry_gb =  self.items2.groupby('category name').apply(lambda x:api_gb_func(x))\
            .sort_values('Count', ascending=False)
        cateogry_gb
        
    def vendor_category_pivot(self):
        self.items2['1'] = 1
        self.items2_sm = self.items2[self.items2['vendor name'].isin(top_vendors)]
        top_vendor_categories = pd.pivot_table(self.items2_sm, index = 'category name', columns='vendor name', values='1'
                      , aggfunc = np.sum, margins=True)\
            .sort_values('All', ascending=False).head(10)
        top_vendor_categories
        
    def excute(self):
        self.create_con()
        self.check_db_stats()
        self.get_items_
        
    
    

In [ ]:
alayzer1 = 

## Peaking at the Structure

In [2]:
con = sqlite3.connect('dealnews.db')
cursor = con.cursor()

tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
tables = [x[0] for x in tables]
tables

['Category Info', 'Category Summary', 'Dealnews Items']

In [3]:
cursor.execute("pragma table_info(`dealnews items`)").fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'headline', 'TEXT', 0, None, 0),
 (2, 'brief_headline', 'TEXT', 0, None, 0),
 (3, 'summary', 'TEXT', 0, None, 0),
 (4, 'secondary_summary', 'TEXT', 0, None, 0),
 (5, 'key_attribute', 'TEXT', 0, None, 0),
 (6, 'brief_notes', 'TEXT', 0, None, 0),
 (7, 'url', 'TEXT', 0, None, 0),
 (8, 'display_date', 'TEXT', 0, None, 0),
 (9, 'publish_datetime_ts', 'TEXT', 0, None, 0),
 (10, 'update_datetime_ts', 'TEXT', 0, None, 0),
 (11, 'expiration_datetime_ts', 'TEXT', 0, None, 0),
 (12, 'last_verified_datetime_ts', 'TEXT', 0, None, 0),
 (13, 'editors_choice', 'INTEGER', 0, None, 0),
 (14, 'sponsored', 'INTEGER', 0, None, 0),
 (15, 'expired', 'INTEGER', 0, None, 0),
 (16, 'expires_today', 'INTEGER', 0, None, 0),
 (17, 'exclusive', 'INTEGER', 0, None, 0),
 (18, 'searchable', 'INTEGER', 0, None, 0),
 (19, 'hotness', 'INTEGER', 0, None, 0),
 (20, 'call_out', 'TEXT', 0, None, 0),
 (21, 'call_out_comparison', 'TEXT', 0, None, 0),
 (22, 'sub_call_out', 'TEXT', 0, None

In [4]:
demo = pd.read_sql('Select * from `Dealnews Items` limit 10', con)
demo.head()

,id,headline,brief_headline,summary,secondary_summary,key_attribute,brief_notes,url,display_date,publish_datetime_ts,update_datetime_ts,expiration_datetime_ts,last_verified_datetime_ts,editors_choice,sponsored,expired,expires_today,exclusive,searchable,hotness,call_out,call_out_comparison,sub_call_out,Coupon Code,category_id_0,namepath_0,ancestor_list_0,category_id_1,namepath_1,ancestor_list_1,vendor_id,name,Image path,API Feed,API id_number,_dt_pulled
0,19319075,Zippo Lighters at Amazon,Zippo Lighters at Amazon: + free shipping w/ Prime,"Save about 15% off or more on most Zippo items on offer, including fuel canisters, lighters, and more.","<div class=""snippet summary"" title=""Save&#x20;about&#x20;15&#x25;&#x20;off&#x20;or&#x20;more&#x20;on&#x20;most&#x20;Zippo&#x20;items&#x20;on&#x20;offer,&#x20;including&#x20;fuel&#x20;canisters,&#x20;lighters,&#x20;and&#x20;more.""> <p>Save about 15% off or more on most Zippo items on offer, inclu...",Amazon,Pictured is the Zippo Matte Pocket Lighter for $15.93 ($6 off).,https://www.dealnews.com/Zippo-Lighters-at-Amazon-free-shipping-w-Prime/19319075.html?iref=site-widget,Last verified 1 hr 19 min ago,2021-06-18 16:50:19,2021-06-18 18:02:29,1970-01-01 00:00:00,2021-06-18 16:50:19,0,0,0,0,0,1,5,,,free shipping w/ Prime,None,197,None,196,None,None,None,313,Amazon,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-18/Zippo Lighters at Amazon_19319075.jpg,Popularity Rank : Popularity: 5/5,1774,2021-06-18 18:10
1,19319108,Staples Fitness Clearance Deals,Staples Fitness Clearance Deals: Save on 20 items,"Save on a selection of medicine balls, speed ropes, resistance bands, and more.","<div class=""snippet summary"" title=""Save&#x20;on&#x20;a&#x20;selection&#x20;of&#x20;medicine&#x20;balls,&#x20;speed&#x20;ropes,&#x20;resistance&#x20;bands,&#x20;and&#x20;more.""> <p>Save on a selection of medicine balls, speed ropes, resistance bands, and more. <a target=""_blank"" href=""https://ww...",Staples,"Pictured is the SPRI Xerball 4-Lb. Medicine Ball for $18.13 (low by $19). Shipping is free over $25 for Staples Rewards members. (It's free to join. Plus, you'll receive no minimum free shipping for the first 3 months.)",https://www.dealnews.com/Staples-Fitness-Clearance-Deals-Save-on-20-items-free-shipping/19319108.html?iref=site-widget,Last verified 1 hr 15 min ago,2021-06-18 16:54:14,2021-06-18 18:02:28,1970-01-01 00:00:00,2021-06-18 16:54:14,0,0,0,0,0,1,5,Save on 20 items,,free shipping,None,655,None,211,None,None,None,274,Staples,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-18/Staples Fitness Clearance Deals_19319108.jpg,Popularity Rank : Popularity: 5/5,1774,2021-06-18 18:10
2,19319048,Yardistry Gazebo Mosquito Mesh Kit,Yardistry Gazebo Mosquito Mesh Kit from $250 for members,"While not all the sizes are readily available for comparison, you'd pay $50 more for the 12x12-ft. kit at Home Depot.","<div class=""snippet summary"" title=""While&#x20;not&#x20;all&#x20;the&#x20;sizes&#x20;are&#x20;readily&#x20;available&#x20;for&#x20;comparison,&#x20;you&#x27;d&#x20;pay&#x20;&#x24;50&#x20;more&#x20;for&#x20;the&#x20;12x12-ft.&#x20;kit&#x20;at&#x20;Home&#x20;Depot.""> <p>While not all the sizes are...",Costco,Free shipping. Non-members pay a 5% surcharge. Model: YM12938.,https://www.dealnews.com/products/Yardistry/Yardistry-Gazebo-Mosquito-Mesh-Kit/230813.html?iref=site-widget,Last verified 2 hr 1 min ago,2021-06-18 16:08:26,2021-06-18 18:02:35,1970-01-01 00:00:00,2021-06-18 16:08:25,0,0,0,0,0,1,5,from $250 for members,,free shipping,None,744,None,"196,198",None,None,None,897,Costco,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-18/Yardistry Gazebo Mosquito Mesh Kit_19319048.jpg,Popularity Rank : Popularity: 5/5,1774,2021-06-18 18:10
3,19319087,Haliluya 100-Ft. 5-in-1 Expandable Garden Hose Set,Haliluya 100-Ft. 5-in-1 Expandable Garden Hose Set for $30,"Apply coupon code ""57QBJPCJ"" for a savings of $30.","<div class=""body-offer body-burst""> <div class=""snippet summary"" title=""Apply&

In [11]:
num_deals = cursor.execute("select count(*) from `Dealnews Items`").fetchall()[0][0]
print(num_deals)
# assert(num_deals > 200000), f"Too many for memory? @ {num_deals}"

66078


In [12]:
items = pd.read_sql('Select * from `Dealnews Items`', con)
items.head()

,id,headline,brief_headline,summary,secondary_summary,key_attribute,brief_notes,url,display_date,publish_datetime_ts,update_datetime_ts,expiration_datetime_ts,last_verified_datetime_ts,editors_choice,sponsored,expired,expires_today,exclusive,searchable,hotness,call_out,call_out_comparison,sub_call_out,Coupon Code,category_id_0,namepath_0,ancestor_list_0,category_id_1,namepath_1,ancestor_list_1,vendor_id,name,Image path,API Feed,API id_number,_dt_pulled
0,19319075,Zippo Lighters at Amazon,Zippo Lighters at Amazon: + free shipping w/ Prime,"Save about 15% off or more on most Zippo items on offer, including fuel canisters, lighters, and more.","<div class=""snippet summary"" title=""Save&#x20;about&#x20;15&#x25;&#x20;off&#x20;or&#x20;more&#x20;on&#x20;most&#x20;Zippo&#x20;items&#x20;on&#x20;offer,&#x20;including&#x20;fuel&#x20;canisters,&#x20;lighters,&#x20;and&#x20;more.""> <p>Save about 15% off or more on most Zippo items on offer, inclu...",Amazon,Pictured is the Zippo Matte Pocket Lighter for $15.93 ($6 off).,https://www.dealnews.com/Zippo-Lighters-at-Amazon-free-shipping-w-Prime/19319075.html?iref=site-widget,Last verified 1 hr 19 min ago,2021-06-18 16:50:19,2021-06-18 18:02:29,1970-01-01 00:00:00,2021-06-18 16:50:19,0,0,0,0,0,1,5,,,free shipping w/ Prime,None,197,None,196,None,None,None,313.0,Amazon,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-18/Zippo Lighters at Amazon_19319075.jpg,Popularity Rank : Popularity: 5/5,1774,2021-06-18 18:10
1,19319108,Staples Fitness Clearance Deals,Staples Fitness Clearance Deals: Save on 20 items,"Save on a selection of medicine balls, speed ropes, resistance bands, and more.","<div class=""snippet summary"" title=""Save&#x20;on&#x20;a&#x20;selection&#x20;of&#x20;medicine&#x20;balls,&#x20;speed&#x20;ropes,&#x20;resistance&#x20;bands,&#x20;and&#x20;more.""> <p>Save on a selection of medicine balls, speed ropes, resistance bands, and more. <a target=""_blank"" href=""https://ww...",Staples,"Pictured is the SPRI Xerball 4-Lb. Medicine Ball for $18.13 (low by $19). Shipping is free over $25 for Staples Rewards members. (It's free to join. Plus, you'll receive no minimum free shipping for the first 3 months.)",https://www.dealnews.com/Staples-Fitness-Clearance-Deals-Save-on-20-items-free-shipping/19319108.html?iref=site-widget,Last verified 1 hr 15 min ago,2021-06-18 16:54:14,2021-06-18 18:02:28,1970-01-01 00:00:00,2021-06-18 16:54:14,0,0,0,0,0,1,5,Save on 20 items,,free shipping,None,655,None,211,None,None,None,274.0,Staples,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-18/Staples Fitness Clearance Deals_19319108.jpg,Popularity Rank : Popularity: 5/5,1774,2021-06-18 18:10
2,19319048,Yardistry Gazebo Mosquito Mesh Kit,Yardistry Gazebo Mosquito Mesh Kit from $250 for members,"While not all the sizes are readily available for comparison, you'd pay $50 more for the 12x12-ft. kit at Home Depot.","<div class=""snippet summary"" title=""While&#x20;not&#x20;all&#x20;the&#x20;sizes&#x20;are&#x20;readily&#x20;available&#x20;for&#x20;comparison,&#x20;you&#x27;d&#x20;pay&#x20;&#x24;50&#x20;more&#x20;for&#x20;the&#x20;12x12-ft.&#x20;kit&#x20;at&#x20;Home&#x20;Depot.""> <p>While not all the sizes are...",Costco,Free shipping. Non-members pay a 5% surcharge. Model: YM12938.,https://www.dealnews.com/products/Yardistry/Yardistry-Gazebo-Mosquito-Mesh-Kit/230813.html?iref=site-widget,Last verified 2 hr 1 min ago,2021-06-18 16:08:26,2021-06-18 18:02:35,1970-01-01 00:00:00,2021-06-18 16:08:25,0,0,0,0,0,1,5,from $250 for members,,free shipping,None,744,None,"196,198",None,None,None,897.0,Costco,/home/malcolm/Demo with APIs/Dealnews Images/2021-06-18/Yardistry Gazebo Mosquito Mesh Kit_19319048.jpg,Popularity Rank : Popularity: 5/5,1774,2021-06-18 18:10
3,19319087,Haliluya 100-Ft. 5-in-1 Expandable Garden Hose Set,Haliluya 100-Ft. 5-in-1 Expandable Garden Hose Set for $30,"Apply coupon code ""57QBJPCJ"" for a savings of $30.","<div class=""body-offer body-burst""> <div class=""snippet summary"" title=""

In [13]:
items.isna().sum()

id                               0
headline                         0
brief_headline                   0
summary                          0
secondary_summary                0
key_attribute                    0
brief_notes                      0
url                              0
display_date                     0
publish_datetime_ts              0
update_datetime_ts               0
expiration_datetime_ts           0
last_verified_datetime_ts        0
editors_choice                   0
sponsored                        0
expired                          0
expires_today                    0
exclusive                        0
searchable                       0
hotness                          0
call_out                         0
call_out_comparison              0
sub_call_out                  4302
Coupon Code                  45500
category_id_0                    0
namepath_0                   66078
ancestor_list_0               8887
category_id_1                60678
namepath_1          

## Parse Callout (Prices and comparisons) 

In [14]:
items[items['call_out'] == ''].shape
items[items['call_out_comparison'] == ''].shape

(19587, 36)

In [15]:
def parse_callout(raw_str):
    str1 = raw_str.replace('$', '').replace(',', '')
    try:
        out = float(str1)
    except:
        out = np.nan
    return(out)

In [17]:
items['callout_parse'] = items['call_out'].apply(parse_callout)
items['callout_compare_parse'] = items['call_out_comparison'].apply(parse_callout)
parsed_cnt = items['callout_parse'].notnull().sum()
parsed_pct = np.round(100*parsed_cnt/items.shape[0], 2)
parse_compare_cnt = items['callout_compare_parse'].notnull().sum()
print(f"Parsed: {parsed_cnt}, Parsed Pct: {parsed_pct}, Parse compare cnt: {parse_compare_cnt}")

Parsed: 44325, Parsed Pct: 67.08, Parse compare cnt: 46487


In [18]:
items['callout_diff'] = items['callout_compare_parse'] - items['callout_parse'] 
items['callout_diff_pct'] = np.round(100*(items['callout_compare_parse'] - items['callout_parse'])
                                     /items['callout_compare_parse'], 2)
pd.concat([items['callout_diff'].describe(), items['callout_diff_pct'].describe()],axis=1)

,callout_diff,callout_diff_pct
count,42258.000000,42258.000000
mean,71.003167,41.874419
std,157.727094,18.825568
min,0.010000,0.140000
25%,10.010000,27.270000
50%,25.000000,41.180000
75%,65.000000,53.570000
max,7000.000000,99.850000


In [19]:
## Take a peak at one or 2
items.sort_values('publish_datetime_ts', ascending=False).head(10) # looking for url to compare

,id,headline,brief_headline,summary,secondary_summary,key_attribute,brief_notes,url,display_date,publish_datetime_ts,update_datetime_ts,expiration_datetime_ts,last_verified_datetime_ts,editors_choice,sponsored,expired,expires_today,exclusive,searchable,hotness,call_out,call_out_comparison,sub_call_out,Coupon Code,category_id_0,namepath_0,ancestor_list_0,category_id_1,namepath_1,ancestor_list_1,vendor_id,name,Image path,API Feed,API id_number,_dt_pulled,callout_compare_parse,callout_parse,callout_diff,callout_diff_pct
66051,20121791,"M-D Building Products 1/8"" x 90-Ft. Caulking Cord","M-D Building Products 1/8"" x 90-Ft. Caulking Cord for $6",It's $9 under list price.,"<div class=""snippet summary"" title=""You&#x27;d&#x20;pay&#x20;&#x24;10&#x20;at&#x20;your&#x20;local&#x20;hardware&#x20;store.""> <p>You'd pay $10 at your local hardware store. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.html?20,2,20121791,iref=site-widget"">Buy Now at Amazon</a> </p>...",Amazon,Free shipping w/ prime. <br><b>Features:</b> for doors and window panes. Model: ‎71548.,https://www.dealnews.com/products/M-D-Building-Products/M-D-Building-Products-1-8-x-90-Ft-Caulking-Cord/300959.html?iref=site-widget,Last verified 45 min ago,2022-04-15 14:51:59,2022-04-15 15:32:17,1969-12-31 19:00:00,2022-04-15 14:51:58,0,0,0,0,0,1,3,$5.98,$14,free shipping w/ Prime,None,862,None,196,None,None,None,313.0,Amazon,"/mnt/volume-nyc3-01/Dealnews_Images/2022-04-15/M-D Building Products 18"" x 90-Ft. Caulking Cord_20121791.jpg",Popularity Rank : Popularity: 3/5,1780,2022-04-15 15:37,14.0,5.98,8.02,57.29
66041,20121776,Crocs Clearance Sale,Crocs Clearance Sale: Up to 50% off,"Save on sandals, clogs, Jibbitz, and more for the family.","<div class=""snippet summary"" title=""Save&#x20;on&#x20;sandals,&#x20;clogs,&#x20;Jibbitz,&#x20;and&#x20;more&#x20;for&#x20;the&#x20;family.""> <p>Save on sandals, clogs, Jibbitz, and more for the family. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.html?20,2,20121776,iref=site-widget...",Crocs,Free shipping w/ $45. Shipping adds $6.99 or is free with orders over $44.99.,https://www.dealnews.com/Crocs-Clearance-Sale-Up-to-50-off-free-shipping-w-45/20121776.html?iref=site-widget,Last verified 48 min ago,2022-04-15 14:48:30,2022-04-15 15:32:17,1969-12-31 19:00:00,2022-04-15 14:48:29,0,0,0,0,0,1,4,Up to 50% off,,free shipping w/ $45,None,280,None,202,None,None,None,5796.0,Crocs,/mnt/volume-nyc3-01/Dealnews_Images/2022-04-15/Crocs Clearance Sale_20121776.jpg,Popularity Rank : Popularity: 4/5,1777,2022-04-15 15:37,NaN,NaN,NaN,NaN
66050,20121779,Eahthni Cable Management Tray 2-Pack,Eahthni Cable Management Tray 2-Pack for $10,"Apply coupon code ""807YO9KL"" for a savings of $40.","<div class=""body-offer body-burst""> <div class=""snippet summary"" title=""Apply&#x20;coupon&#x20;code&#x20;&quot;807YO9KL&quot;&#x20;for&#x20;a&#x20;savings&#x20;of&#x20;&#x24;40.""> <p>Apply coupon code ""807YO9KL"" for a savings of $40. <a target=""_blank"" href=""https://www.dealnews.com/lw/click.htm...",Amazon,"Free shipping. It may take up to seven weeks to arrive. <br><b>Features:</b> powder-coated iron, 17"" and 12"" trays, 10 tapes, and 10 organizers and mounting hardware included.",https://www.dealnews.com/Eahthni-Cable-Management-Tray-2-Pack-for-10-free-shipping/20121779.html?iref=site-widget,Last verified 49 min ago,2022-04-15 14:48:07,2022-04-15 15:32:18,2022-05-10 12:33:00,2022-04-15 14:48:06,0,0,0,0,0,1,4,$9.99,$50,free shipping,807YO9KL,89,None,39,None,None,None,313.0,Amazon,/mnt/volume-nyc3-01/Dealnews_Images/2022-04-15/Eahthni Cable Management Tray 2-Pack_20121779.jpg,Popularity Rank : Popularity: 4/5,1777,2022-04-15 15:37,50.0,9.99,40.01,80.02
66037,20121758,Southwire Romex SIMpull 25-Foot 14/2 NM-B Indoor Electrical Copper Wire,Southwire Romex SIMpull 25-Foot 14/2 NM-B Indoor Electrical Copper Wire for $16,"Like a bird on the wire, Like a drunk in a midnight choir, It has tried in its way to be free. (Well, it didn'

In [14]:
items[items['call_out'] == ''].shape
items[items['call_out_comparison'] == ''].shape

(19587, 36)

### Filter for dups and items with callout numbers

In [43]:
items_sm = items.drop_duplicates(subset=['headline'])
items_sm = items_sm[items_sm['callout_parse'].notnull()]
print(f"Keep {items_sm.shape[0]} ", np.round(100*items_sm.shape[0]/items.shape[0],2))
items_sm.shape

Keep 29636  44.85


(29636, 41)

In [21]:
items_sm.groupby('API Feed')['headline'].count()

API Feed
Popularity Rank : Popularity: 1/5    3224
Popularity Rank : Popularity: 2/5    6004
Popularity Rank : Popularity: 3/5    9414
Popularity Rank : Popularity: 4/5    8445
Popularity Rank : Popularity: 5/5    2549
Name: headline, dtype: int64

## Get Category Lookup table

In [22]:
cat_0_deals = items_sm.groupby('category_id_0')['headline'].count().sort_values(ascending=False)
cat_0_deals = cat_0_deals.reset_index().rename({'headline':'# of deals'}, axis=1)
cat_0_deals

,category_id_0,# of deals
0,280,1412
1,226,1187
2,201,1173
3,49,811
4,197,695
...,...,...
344,209,1
345,210,1
346,1057,1
347,299,1


In [23]:
last_category_date = cursor.execute("select max(_dt_pulled) from `Category Info`")
last_category_date = last_category_date.fetchall()[0][0]
print(last_category_date)
categories = pd.read_sql(f"select * from `Category Info` where _dt_pulled='{last_category_date}' ", con)
categories

2022-04-15 15:37


,level_0,level_1,name,count,id_number,short_name,id_name,_dt_pulled
0,vendor,0,Amazon,854,313,Amazon,vendor_id,2022-04-15 15:37
1,vendor,1,eBay,152,50,eBay,vendor_id,2022-04-15 15:37
2,vendor,2,Macy's,64,288,Macy's,vendor_id,2022-04-15 15:37
3,vendor,3,Walmart,63,321,Walmart,vendor_id,2022-04-15 15:37
4,vendor,4,Home Depot,44,958,Home Depot,vendor_id,2022-04-15 15:37
...,...,...,...,...,...,...,...,...
881,category,21,Groceries,1,214,Groceries,category_id,2022-04-15 15:37
882,category,22,Toys & Hobbies,1,226,Toys & Hobbies,category_id,2022-04-15 15:37
883,category,23,Bed & Bath,1,360,Bed & Bath,category_id,2022-04-15 15:37
884,category,24,Computer Services,1,440,Computer Services,category_id,2022-04-15 15:37


In [24]:
category_ids_sm = categories[categories['level_0'] == 'category']
category_ids_sm = category_ids_sm[['id_number', 'name', 'short_name']]
category_ids_sm

,id_number,name,short_name
860,196,Home & Garden,Home & Garden
861,202,Clothing & Accessories,Clothing & Accessories
862,142,Electronics,Electronics
863,39,Computers,Computers
864,211,Sports & Fitness,Sports & Fitness
865,40,Store Events,Store Events
866,756,Health & Beauty,Health & Beauty
867,186,Gaming & Toys,Gaming & Toys
868,182,Office & School Supplies,Office & School Supplies
869,238,Automotive,Automotive


In [25]:
pd.merge(cat_0_deals, category_ids_sm, left_on ='category_id_0', right_on='id_number')

,category_id_0,# of deals,id_number,name,short_name
0,226,1187,226,Toys & Hobbies,Toys & Hobbies
1,238,618,238,Automotive,Automotive
2,196,591,196,Home & Garden,Home & Garden
3,214,292,214,Groceries,Groceries
4,360,254,360,Bed & Bath,Bed & Bath
5,211,209,211,Sports & Fitness,Sports & Fitness
6,75,204,75,Monitors,Monitors
7,744,148,744,Patio Furniture,Patio Furniture
8,202,90,202,Clothing & Accessories,Clothing & Accessories
9,142,59,142,Electronics,Electronics


In [26]:
items.columns

Index(['id', 'headline', 'brief_headline', 'summary', 'secondary_summary',
       'key_attribute', 'brief_notes', 'url', 'display_date',
       'publish_datetime_ts', 'update_datetime_ts', 'expiration_datetime_ts',
       'last_verified_datetime_ts', 'editors_choice', 'sponsored', 'expired',
       'expires_today', 'exclusive', 'searchable', 'hotness', 'call_out',
       'call_out_comparison', 'sub_call_out', 'Coupon Code', 'category_id_0',
       'namepath_0', 'ancestor_list_0', 'category_id_1', 'namepath_1',
       'ancestor_list_1', 'vendor_id', 'name', 'Image path', 'API Feed',
       'API id_number', '_dt_pulled', 'callout_compare_parse', 'callout_parse',
       'callout_diff', 'callout_diff_pct'],
      dtype='object')

## Callout Differences

In [55]:
def api_gb_func(df):
    out = {}
    out['Count'] = df.shape[0]
    out['# w/ $ callout'] = df['callout_parse'].notnull().sum()
    if df.shape[0] > 1:
        out['Avg Callout Price'] = np.round(df['callout_parse'].mean(), 2)
        out['Avg Callout Pct Diff'] = np.round(df['callout_diff_pct'].mean(), 2)
    else:
        out['Avg Callout Price'] = np.nan
        out['Avg Callout Pct Diff'] = np.nan
    
    return(pd.Series(out))

In [28]:
api_feed_gb = items_sm.groupby('API Feed').apply(lambda x:api_gb_func(x))
# api_feed_gb['Pct w callout'] = np.round(100*api_feed_gb['# w/ $ callout']/api_feed_gb['Count'], 2) 
api_feed_gb['Count Ratio to Max'] = np.round(api_feed_gb['Count']/api_feed_gb['Count'].max(),2)
api_feed_gb

,Count,# w/ $ callout,Avg Callout Price,Avg Callout Pct Diff,Pct w callout,Count Ratio to Max
API Feed,,,,,,
Popularity Rank : Popularity: 1/5,3224.0,3224.0,162.94,35.82,100.0,0.34
Popularity Rank : Popularity: 2/5,6004.0,6004.0,164.96,36.39,100.0,0.64
Popularity Rank : Popularity: 3/5,9414.0,9414.0,103.78,41.22,100.0,1.00
Popularity Rank : Popularity: 4/5,8445.0,8445.0,79.59,44.48,100.0,0.90
Popularity Rank : Popularity: 5/5,2549.0,2549.0,58.71,48.68,100.0,0.27


In [29]:
items2 = pd.merge(items_sm, category_ids_sm
                  , left_on = 'category_id_0', right_on='id_number'
                  , how='left').drop(['id_number', 'short_name'], axis=1)\
    .rename({'name_x':'vendor name', 
             'name_y':'category name'}, axis=1)
items2.shape

(29636, 41)

In [30]:
items2.columns

Index(['id', 'headline', 'brief_headline', 'summary', 'secondary_summary',
       'key_attribute', 'brief_notes', 'url', 'display_date',
       'publish_datetime_ts', 'update_datetime_ts', 'expiration_datetime_ts',
       'last_verified_datetime_ts', 'editors_choice', 'sponsored', 'expired',
       'expires_today', 'exclusive', 'searchable', 'hotness', 'call_out',
       'call_out_comparison', 'sub_call_out', 'Coupon Code', 'category_id_0',
       'namepath_0', 'ancestor_list_0', 'category_id_1', 'namepath_1',
       'ancestor_list_1', 'vendor_id', 'vendor name', 'Image path', 'API Feed',
       'API id_number', '_dt_pulled', 'callout_compare_parse', 'callout_parse',
       'callout_diff', 'callout_diff_pct', 'category name'],
      dtype='object')

In [31]:
vendor_gb = items2.groupby('vendor name').apply(lambda x:api_gb_func(x))\
    .sort_values('Count', ascending=False)
vendor_gb.head(12)

,Count,# w/ $ callout,Avg Callout Price,Avg Callout Pct Diff
vendor name,,,,
Amazon,15448.0,15448.0,63.41,39.86
eBay,2781.0,2781.0,225.26,40.57
Macy's,1179.0,1179.0,64.67,61.19
Home Depot,952.0,952.0,208.72,28.18
Walmart,718.0,718.0,112.54,40.33
Woot! An Amazon Company,424.0,424.0,136.72,45.18
Lowe's,356.0,356.0,207.36,31.40
Wayfair,277.0,277.0,170.39,39.72
HP,248.0,248.0,642.87,19.99


In [32]:
top_vendors = vendor_gb.sort_values('Count', ascending=False)[:10].index.tolist()
top_vendors

['Amazon',
 'eBay',
 "Macy's",
 'Home Depot',
 'Walmart',
 'Woot! An Amazon Company',
 "Lowe's",
 'Wayfair',
 'HP',
 'Nike']

In [33]:
cateogry_gb =  items2.groupby('category name').apply(lambda x:api_gb_func(x))\
    .sort_values('Count', ascending=False)
cateogry_gb

,Count,# w/ $ callout,Avg Callout Price,Avg Callout Pct Diff
category name,,,,
Toys & Hobbies,1187.0,1187.0,75.13,40.11
Automotive,618.0,618.0,47.48,38.20
Home & Garden,591.0,591.0,54.86,37.80
Groceries,292.0,292.0,17.26,36.03
Bed & Bath,254.0,254.0,116.88,38.95
Sports & Fitness,209.0,209.0,123.52,39.46
Monitors,204.0,204.0,276.49,29.05
Patio Furniture,148.0,148.0,237.41,31.92
Clothing & Accessories,90.0,90.0,23.31,52.29


In [34]:
items2['1'] = 1
items2_sm = items2[items2['vendor name'].isin(top_vendors)]
top_vendor_categories = pd.pivot_table(items2_sm, index = 'category name', columns='vendor name', values='1'
              , aggfunc = np.sum, margins=True)\
    .sort_values('All', ascending=False).head(10)
top_vendor_categories

vendor name,Amazon,HP,Home Depot,Lowe's,Macy's,Nike,Walmart,Wayfair,Woot! An Amazon Company,eBay,All
category name,,,,,,,,,,,
All,2575.0,24.0,36.0,34.0,52.0,4.0,131.0,56.0,70.0,224.0,3206
Toys & Hobbies,833.0,NaN,NaN,2.0,28.0,NaN,70.0,6.0,18.0,81.0,1038
Automotive,480.0,NaN,3.0,2.0,NaN,NaN,12.0,1.0,8.0,29.0,535
Home & Garden,386.0,NaN,13.0,9.0,1.0,NaN,18.0,26.0,11.0,20.0,484
Groceries,240.0,NaN,NaN,NaN,1.0,NaN,5.0,NaN,5.0,NaN,251
Bed & Bath,176.0,NaN,11.0,10.0,7.0,NaN,2.0,11.0,8.0,6.0,231
Sports & Fitness,136.0,NaN,NaN,1.0,4.0,2.0,4.0,1.0,4.0,21.0,173
Monitors,76.0,24.0,NaN,NaN,NaN,NaN,3.0,NaN,6.0,40.0,149
Patio Furniture,54.0,NaN,7.0,6.0,NaN,NaN,6.0,10.0,3.0,9.0,95


In [35]:
amazon_items = items2[items2['vendor name'] == 'Amazon']
non_amazon_items = items2[items2['vendor name'] != 'Amazon']

In [36]:
amzn_categories = pd.concat({'Count' : amazon_items['category name'].value_counts()
                            ,'Percent' :np.round(100*amazon_items['category name']\
                                                     .value_counts(normalize=True), 2)}
                            , axis=1)
amzn_categories.head(10)

,Count,Percent
Toys & Hobbies,833,32.35
Automotive,480,18.64
Home & Garden,386,14.99
Groceries,240,9.32
Bed & Bath,176,6.83
Sports & Fitness,136,5.28
Monitors,76,2.95
Patio Furniture,54,2.10
Electronics,47,1.83
Office & School Supplies,33,1.28


In [37]:
amzn_categories = amazon_items.groupby('category name').apply(lambda x: api_gb_func(x))\
    .sort_values('Avg Callout Pct Diff', ascending=False)
amzn_categories[amzn_categories['Count'] > 20]

,Count,# w/ $ callout,Avg Callout Price,Avg Callout Pct Diff
category name,,,,
Toys & Hobbies,833.0,833.0,46.62,40.97
Bed & Bath,176.0,176.0,52.66,39.71
Sports & Fitness,136.0,136.0,81.44,39.14
Electronics,47.0,47.0,43.00,38.50
Automotive,480.0,480.0,42.02,38.21
Home & Garden,386.0,386.0,36.66,36.31
Patio Furniture,54.0,54.0,167.91,35.42
Office & School Supplies,33.0,33.0,39.49,34.99
Groceries,240.0,240.0,11.82,34.88


In [38]:
non_amzn_categories = non_amazon_items.groupby('category name').apply(lambda x: api_gb_func(x))\
    .sort_values('Avg Callout Pct Diff', ascending=False)
non_amzn_categories[non_amzn_categories['Count'] > 20]

,Count,# w/ $ callout,Avg Callout Price,Avg Callout Pct Diff
category name,,,,
Clothing & Accessories,69.0,69.0,22.02,58.02
Groceries,52.0,52.0,42.32,42.22
Home & Garden,205.0,205.0,89.11,40.74
Sports & Fitness,73.0,73.0,201.91,40.08
Automotive,138.0,138.0,66.46,38.17
Toys & Hobbies,354.0,354.0,142.21,37.97
Bed & Bath,78.0,78.0,261.79,37.22
Patio Furniture,94.0,94.0,277.34,29.94
Monitors,128.0,128.0,267.48,28.83


## Over time analysis

In [46]:
import plotly.express as px 

In [52]:
items_sm = items.drop_duplicates(subset=['headline'])
items_sm = items_sm[items_sm['callout_parse'].notnull()]
print(f"Keep {items_sm.shape[0]} ", np.round(100*items_sm.shape[0]/items.shape[0],2))
items_sm['publish_time'] = pd.to_datetime(items_sm['publish_datetime_ts'])


Keep 29636  44.85


In [74]:
items_sm_time = items_sm.set_index('publish_time') 
time_gb = items_sm_time.groupby(pd.Grouper(freq='D')).apply(lambda x:api_gb_func(x))\
    .reset_index()
time_gb = time_gb[time_gb['Count'] > 0]\
    .sort_values('publish_time')\
    .tail(180) # Last 6  months
time_gb['DOW'] = time_gb['publish_time'].dt.day_name()
time_gb['Weekend_flag'] = time_gb['DOW'].isin(['Saturday','Sunday'])
time_gb

,publish_time,Count,# w/ $ callout,Avg Callout Price,Avg Callout Pct Diff,DOW,Weekend_flag
234,2021-10-18,153.0,153.0,136.15,37.89,Monday,False
235,2021-10-19,145.0,145.0,138.24,40.16,Tuesday,False
236,2021-10-20,129.0,129.0,140.21,41.11,Wednesday,False
237,2021-10-21,125.0,125.0,150.78,38.95,Thursday,False
238,2021-10-22,126.0,126.0,201.06,37.24,Friday,False
239,2021-10-23,29.0,29.0,76.64,38.32,Saturday,True
240,2021-10-24,31.0,31.0,71.32,37.97,Sunday,True
241,2021-10-25,140.0,140.0,86.73,40.86,Monday,False
242,2021-10-26,130.0,130.0,101.47,37.66,Tuesday,False
243,2021-10-27,124.0,124.0,83.28,40.87,Wednesday,False


In [73]:
px.scatter(time_gb, 'publish_time', 'Count'
           , color='Weekend_flag'
          , trendline='ols')

In [76]:
px.scatter(time_gb, 'publish_time', 'Avg Callout Price'
          , trendline='ols')

In [77]:
px.scatter(time_gb, 'publish_time', 'Avg Callout Pct Diff'
          , trendline='ols')

In [39]:
items['_dt_pulled'].max()

'2022-04-15 15:37'

In [40]:
items['_dt_pulled'].min()

'2021-06-18 18:10'

In [41]:
items['date'] = items['_dt_pulled'].str[:10]